In [1]:
# !pip install loguru
# !pip install jsonpath_ng

You should consider upgrading via the '/root/miniconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/miniconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import os
import json
from loguru import logger as log
import traceback
from jsonpath_ng.ext import parse
import pickle

## API Simulation

Features

enrollee_id : Unique ID for candidate

city: City code

city_ development _index : Developement index of the city (scaled)

relevent_experience: Relevant experience of candidate

enrolled_university: Type of University course enrolled if any

education_level: Education level of candidate

major_discipline :Education major discipline of candidate

experience: Candidate total experience in years

company_size: No of employees in current employer's company

company_type : Type of current employer

lastnewjob: Difference in years between previous job and current job

In [3]:
def extract_field(jpath, data):
    return [match.value for match in parse(jpath).find(data)]

def convert_single_data_JPATH(jpath, data):
    declared_value_field = extract_field(jpath, data)
    return declared_value_field if len(declared_value_field) else 'None'

def escoragem_api(data_json):
    try:
        df_prediction = define_policy(data_json)
        JSONP_data = df_prediction[["SCORE","RATING","Probability - Looking for job change expectations"]].to_json(orient='records')
        return JSONP_data
    except Exception as exp:
        log.error(exp)
        log.error(data_json)
        traceback.print_exc()
        
def extract_features(PAYLOAD_DATA):
    enrollee_id = convert_single_data_JPATH(enrollee_id_JPATH, PAYLOAD_DATA)
    city = convert_single_data_JPATH(city_JPATH, PAYLOAD_DATA)
    city_development_index = convert_single_data_JPATH(city_development_index_JPATH, PAYLOAD_DATA)
    relevent_experience = convert_single_data_JPATH(relevent_experience_JPATH, PAYLOAD_DATA)
    enrolled_university = convert_single_data_JPATH(enrolled_university_JPATH, PAYLOAD_DATA)
    education_level = convert_single_data_JPATH(education_level_JPATH, PAYLOAD_DATA)
    major_discipline = convert_single_data_JPATH(major_discipline_JPATH, PAYLOAD_DATA)
    experience = convert_single_data_JPATH(experience_JPATH, PAYLOAD_DATA)
    company_size = convert_single_data_JPATH(company_size_JPATH, PAYLOAD_DATA)
    company_type = convert_single_data_JPATH(company_type_JPATH, PAYLOAD_DATA)
    lastnewjob = convert_single_data_JPATH(lastnewjob_JPATH, PAYLOAD_DATA)
   
    dict_vars = {'enrollee_id':enrollee_id,'city':city,
                 'city_development_index':city_development_index,'relevent_experience':relevent_experience,
                 'enrolled_university':enrolled_university,'education_level':education_level,
                 'major_discipline':major_discipline,'experience':experience,
                 'company_size':company_size,
                 'company_type':company_type,
                 'last_new_job':lastnewjob}
    df_hr = pd.DataFrame(dict_vars)
    return df_hr
        
def define_rating(df):
    df["RATING"] = ""
    df.loc[(df['SCORE']>=0.0) & (df['SCORE']<300) ,"RATING"] = 'E'
    df.loc[(df['SCORE']>=300) & (df['SCORE']<500) ,"RATING"] = 'D'
    df.loc[(df['SCORE']>=500) & (df['SCORE']<700) ,"RATING"] = 'C'
    df.loc[(df['SCORE']>=700) & (df['SCORE']<950) ,"RATING"] = 'B'
    df.loc[(df['SCORE']>=950) & (df['SCORE']<1000) ,"RATING"] = 'A'

    df["Probability - Looking for job change expectations"] = ""
    df.loc[(df['SCORE']>=0.0) & (df['SCORE']<300) ,"Probability - Looking for job change expectations"] = '64.84%'    
    df.loc[(df['SCORE']>=300) & (df['SCORE']<500) ,"Probability - Looking for job change expectations"] = '59.26%'
    df.loc[(df['SCORE']>=500) & (df['SCORE']<700) ,"Probability - Looking for job change expectations"] = '45.40%'
    df.loc[(df['SCORE']>=700) & (df['SCORE']<950) ,"Probability - Looking for job change expectations"] = '10.13%'
    df.loc[(df['SCORE']>=950) & (df['SCORE']<1000) ,"Probability - Looking for job change expectations"] = '7.03%'
    return df

def score_model(data_payload):
    loaded_model = pickle.load(open(MODEL_PATH_CAT, 'rb'))
    pred = loaded_model.predict_proba(data_payload)[:, 1]
    return (1-pred)*1000

def define_policy(data_json):
    try:
        prediction_df = extract_features(data_json)
        score = score_model(prediction_df[VARS_MODEL])
        prediction_df['SCORE'] = score
        policy_df = define_rating(prediction_df)
        return policy_df
    except Exception as exp:
        log.error(exp)
        traceback.print_exc()

        

In [4]:
# current_file = os.path.abspath(os.path.dirname(__file__))
# path_root = str((Path(current_file).parent / 'static').resolve())

path_root = "../static/"
json_file_path = path_root+"json_example.json"

with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
     json_payload = json.load(json_data, strict=False)
json_payload        

{'Result': [{'enrollee_id': 32403,
   'city': 'city_41',
   'city_development_index': 0.827,
   'relevent_experience': 'Has relevent experience',
   'enrolled_university': 'Full time course',
   'education_level': 'Graduate',
   'major_discipline': '<10',
   'experience': '>10',
   'company_size': 'Pvt Ltd',
   'company_type': '<10',
   'last_new_job': '>4'}]}

In [5]:
enrollee_id_JPATH = "$.Result[0].enrollee_id"
city_JPATH = "$.Result[0].city"
city_development_index_JPATH = "$.Result[0].city_development_index"
relevent_experience_JPATH = "$.Result[0].relevent_experience"
enrolled_university_JPATH = "$.Result[0].enrolled_university"
education_level_JPATH = "$.Result[0].education_level"
major_discipline_JPATH = "$.Result[0].major_discipline"
experience_JPATH = "$.Result[0].experience"
company_size_JPATH = "$.Result[0].company_size"
company_type_JPATH = "$.Result[0].company_type"
lastnewjob_JPATH = "$.Result[0].lastnewjob"

VARS_MODEL = ['city','city_development_index','relevent_experience',
              'enrolled_university','education_level','major_discipline','experience','company_size',
              'company_type','last_new_job']

MODEL_PATH_CAT = path_root+"model_HRAnalytics.pkl"

In [6]:
teste_df = pd.read_csv("../static/example_csv.csv")
teste_df.head()

enrollee_id      city  city_development_index      relevent_experience  \
0        12581  city_104                   0.924  Has relevent experience   
1         6444  city_114                   0.926  Has relevent experience   
2        17221   city_21                   0.624   No relevent experience   
3        20635   city_21                   0.624   No relevent experience   
4        19794   city_16                   0.910   No relevent experience   

  enrolled_university education_level major_discipline  experience  \
0    Full time course        Graduate             STEM           3   
1       no_enrollment        Graduate             STEM          16   
2    Full time course        Graduate             STEM           4   
3    Part time course        Graduate             STEM           1   
4    Full time course        Graduate             STEM           5   

  company_size   company_type last_new_job  
0          <10        Pvt Ltd        never  
1    1000-4999        Pvt Ltd            4  
2          NaN            NaN        never  
3       10000+        Pvt Ltd            1  
4        50-99  Public Sector            2

In [7]:
data_payload = {
  "Result": [
    {
      "enrollee_id": 3243,
      "city": "city_21",
      "city_development_index": 0.926,
      "relevent_experience": "No relevent experience",
      "enrolled_university": "no_enrollment	",
      "education_level": "Graduate",
      "major_discipline": "STEM",
      "experience": ">10",
      "company_size": "STEM",
      "major_discipline": "<3",
      "company_size": "Public Sector",
      "company_type": "10000+",        
      "last_new_job": ">4"
      }
  ]}

In [8]:
json_payload

{'Result': [{'enrollee_id': 32403,
   'city': 'city_41',
   'city_development_index': 0.827,
   'relevent_experience': 'Has relevent experience',
   'enrolled_university': 'Full time course',
   'education_level': 'Graduate',
   'major_discipline': '<10',
   'experience': '>10',
   'company_size': 'Pvt Ltd',
   'company_type': '<10',
   'last_new_job': '>4'}]}

## Run

In [9]:
result1 = escoragem_api(json_payload)
result1

'[{"SCORE":809.3946048736,"RATING":"B","Probability - Looking for job change expectations":"10.13%"}]'

In [10]:
result2 = escoragem_api(data_payload)
result2

'[{"SCORE":949.6585749324,"RATING":"B","Probability - Looking for job change expectations":"10.13%"}]'